<a href="https://colab.research.google.com/github/Beatriz-Yun/AIFFEL_LMS/blob/main/Exploration/%5BE-04%5D%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


우리 주변의 모든 텍스트는 시퀀스 데이터이다.

**시퀀스(sequence)**는 데이터에 순서(번호)를 붙여 나열한 것이다.
- 특정 위치의 데이터를 가리킬 수 있다.
- 정렬되어 있지 않아도 된다.

<br>

---

<br>


문장을 구성하는 단어들은 문법이라는 규칙을 따라 나열되어 있는데,<br>
이를 통해 텍스트 데이터를 예측하는 것은 예외가 너무 많다.


따라서 텍스트데이터는 **통계를 기반으로 예측**한다.<br>
ex) 'I' 다음에는 'am'이 오면 반이상 맞는다고 한다.

<br>

즉, 인공지능이 텍스트를 이해하는 방식은 어떤 문법적인 원리가 아니라 **수많은 텍스트를 읽게 함으로써**<br>
통계적으로 다음에 올 텍스트를 예측하는 것이다.

**RNN(순환신경망)**
- \<start\>토큰을 첫 입력으로 받아 생성이 끝나면 \<end\>토큰을 생성한다.
- **생성한 토큰(단어)을 다시 입력으로 사용**한다.

**언어 모델(Language Model)**: n−1개의 단어 시퀀스 $w_1, \cdots, w_{n-1}$가 주어졌을 때, $n$번째 단어 $w_n$으로 무엇이 올지를 예측하는 확률 모델
- 어떤 텍스트도 언어모델의 학습 데이터가 될 수 있다.
- 테스트할 때는 일정한 단어 시퀀스가 주어지면 다음 단어, 그 다음 단어를 계속해서 예측하며 텍스트를 생성한다.

파라미터$\theta$로 모델링하는 언어모델 표현: $P(w_1, \cdots, w_{n-1};\theta)$

## <연극대사 생성 모델 만들기>

## '데이터 전처리' 과정

### 1. 데이터 살피기

In [3]:
import re 
import numpy as np
import tensorflow as tf

# 파일을 읽기모드로 열고
# 라인 단위로 끊어서 list 형태로 읽어옵니다.
file_path = '/content/drive/MyDrive/Colab Notebooks/AIFFEL_LMS/data/shakespeare.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()

# 앞에서부터 10라인 출력
print(raw_corpus[:9])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


- 화자이름과 공백으로 된 데이터 제거하기

In [4]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인
        
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


### 2. 토큰화(Tokenize): 문장을 일정한 기준으로 쪼갠다.
- 공백을 기준으로 쪼개보자.
 - 문장 부호 양쪽에 공백 추가
 - 모두 소문자로 변환
 - 특수문자 모두 제거

In [17]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 문장부호 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자(공백포함 특수문자)를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()                     # 1
    #print(sentence)
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)     # 2 (정규표현식을 활용한다)
    #print(sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)             # 3
    #print(sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)   # 4
    #print(sentence)
    sentence = sentence.strip()                             # 5
    #print(sentence)
    sentence = '<start> ' + sentence + ' <end>'             # 6
    return sentence


In [18]:
# 문장이 어떻게 필터링되는지 확인 (print문 주석처리 이전)
sent = "This @_is ;;;sample        sentence. ten-year-old?"
print('입력문장->',sent)
print(preprocess_sentence(sent))

입력문장-> This @_is ;;;sample        sentence. ten-year-old?
this @_is ;;;sample        sentence. ten-year-old?
this @_is ;;;sample        sentence .  ten-year-old ? 
this @_is ;;;sample sentence . ten-year-old ? 
this is sample sentence . ten year old ? 
this is sample sentence . ten year old ?
<start> this is sample sentence . ten year old ? <end>


자연어처리 분야에서 모델의 입력이 되는 문장을 소스 문장(**Source Sentence**),<br> 정답 역할을 하게 될 모델의 출력 문장을 타겟 문장(**Target Sentence**) 라고 관례적으로 부른다.

즉, source sentence는 X_train이고, target sentence는 y_train이다.

In [7]:
# 여기에 전처리된 문장을 모은다.
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뛴다. (공백 및 화자이름)
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 전처리함수에 문장을 입력 후 전처리된 문장을 corpus리스트에 추가.
    preprocessed_sentence = preprocess_sentence(sentence)   # (print문 주석처리 이후)
    corpus.append(preprocessed_sentence)
        
# 전처리된 문장 10개 확인
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

### 3. 벡터화(Vectorize): 텍스트데이터를 숫자로 변환
- **tf.keras.preprocessing.text.Tokenizer** 패키지 사용 [(참고1, ](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer) [참고2)](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences)
 - 입력: 전처리된 텍스트
 - 출력: 단어 사전(vocabulary)
- 벡터화된 데이터를 **텐서(tensor)**라고 한다.
 - tensorflow로 만든 모델의 입출력데이터는 모두 텐서로 변환되어 처리된다.

**Tokenizer.fit_on_texts:** Updates internal vocabulary based on a list of texts.
<br>
**Tokenizer.texts_to_sequences:** Transforms each text in texts to a sequence of integers.<br><br>

**pad_sequence의 padding옵션:** padding='post'인 경우, 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰준다. 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용한다.




In [8]:
# 텐서플로우의 Tokenizer와 pad_sequences를 사용하여 토큰화한다.

def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)

    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus) 

    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f0907850890>


**벡터화된 데이터**를 확인해보면 모두 정수로 이루어져 있다.<br>
이 숫자는 **단어사전의 인덱스**이다.<br>
(0은 패딩문자 \<pad\>이다.)

In [9]:
print(tensor[:3, :10])

[[   2  143   40  933  140  591    4  124   24  110]
 [   2  110    4  110    5    3    0    0    0    0]
 [   2   11   50   43 1201  316    9  201   74    9]]


단어사전 확인

In [10]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


### 4. Source sentence와 Target sentence로 분리

In [11]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]

# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


#### 데이터셋 객체 생성
지금까지는 model.fit(x_train, y_train, ...)형태로 ndarray를 생성하여 학습했다.<br>
그러나 tensorflow를 활용할 때는 **tf.data.Dataset 객체**를 생성하는 방법을 흔히 사용한다.<br>
tensorflow에서 사용할 경우 <u>데이터 입력 파이프라인을 통한 속도 개선 및 편의 기능을 제공</u>하므로 꼭 알아두자.<br>

[중요](https://www.tensorflow.org/api_docs/python/tf/data/Dataset)

<br>

데이터셋을 tensor형태로 생성했으므로 **from_tensor_slices()**메소드를 사용하여 **tf.data.Dataset객체**를 생성한다.

In [12]:
BUFFER_SIZE = len(src_input)   # source sentence 수
BATCH_SIZE = 256               # 한 번 학습할 데이터 수
steps_per_epoch = len(src_input) // BATCH_SIZE

# VOCAB_SIZE: 벡터화한 단어 개수
# tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)   # 데이터셋을 배치크기씩 나눈다.
                                                           # drop_remainder=True는 배치크기보다 작은 그룹을 버린다.
dataset

<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

## 모델 구성 및 학습과정

### 모델의 구조도

- tf.keras.Model을 Subclassing(상속)하는 방식으로 만들 것이다.
[(Subclassing Model)](https://www.tensorflow.org/guide/keras/custom_layers_and_models?hl=ko)
- 만들 모델은 1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있다.

![이미지](https://drive.google.com/uc?id=1sjQXo2D9b5p83ypaHjmm6ZU967H4hpA2)

<br><br>

**Embedding레이어:** 단어사전(voocabulary)에 있는 인덱스 값을 <u>해당 인덱스번째의 word vector로 바꿔준다.</u>
- **word vector**는 의미 벡터 공간에서 단어의 추상적 표현(representation)으로 사용된다.



In [13]:
class TextGenerator(tf.keras.Model):
  # embedding_size는 word vector의 차원수이다.
  # hidden_size는 LSTM레이어의 hidden state의 차원수이다.
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

모델의 최종 출력 텐서의 shape이 (256, 20, 7001)임을 확인할 수 있다.
- 7001은 Dense레이어의 출력 차원수이다.
 - 7001개의 단어 중 어느 단어의 확률이 가장 높을지를 모델링해야 하기 때문이다.
- 256은 이전단계에서 지정한 배치 사이즈이다.
 - dataset.take(1)은 데이터셋에서 1개의 배치를 가져온다.
- 20은 LSTM레이어에서 **return_sequences=True**로 지정함으로써 나타난다.
 - LSTM이 **자신에게 입력된 시퀀스의 길이만큼 동일한 길이의 시퀀스를 출력**한다는 의미이다.
 - 즉, 우리의 데이터셋의 max_len이 20이었기 때문이다.

In [14]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[ 1.60345386e-04,  6.09239687e-05,  2.89983233e-04, ...,
          1.42004836e-04, -2.85116927e-04, -1.33140813e-04],
        [ 7.18360941e-04, -4.11577130e-05,  1.03884209e-04, ...,
         -1.40163596e-04, -6.53263938e-04, -4.31908382e-04],
        [ 6.15979428e-04,  1.01792750e-04, -1.49642263e-04, ...,
         -1.00366102e-04, -1.05997943e-03, -6.57598954e-04],
        ...,
        [ 3.78154218e-04, -2.25151097e-03, -2.55316473e-03, ...,
         -3.29263159e-03, -1.43250008e-03,  1.30326510e-03],
        [ 5.38947061e-04, -2.20816443e-03, -2.83878180e-03, ...,
         -3.48402094e-03, -1.51188811e-03,  1.55652477e-03],
        [ 6.96565898e-04, -2.13662582e-03, -3.11367470e-03, ...,
         -3.64306709e-03, -1.58119528e-03,  1.75599975e-03]],

       [[ 1.60345386e-04,  6.09239687e-05,  2.89983233e-04, ...,
          1.42004836e-04, -2.85116927e-04, -1.33140813e-04],
        [ 1.29127075e-04,  2.69704266e-04,  5.

우리의 모델은 입력 시퀀스의 길이를 모르기 때문에 Output Shape을 특정할 수 없다.<br>
따라서 model.summary()를 해도 Ooutput Shape을 정확하게 알려주지 않는다.

In [15]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


### 모델 학습시키기

**tf.test.is_gpu_available()**를 통해 tensorflow가 GPU를 사용하고 있는지 확인할 수 있다.

In [16]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True


[참고1. optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers)


[참고2. loss](https://www.tensorflow.org/api_docs/python/tf/keras/losses)

In [19]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
93/93 [==============================] - 20s 180ms/step - loss: 3.4779
Epoch 2/30
93/93 [==============================] - 17s 182ms/step - loss: 2.8068
Epoch 3/30
93/93 [==============================] - 17s 184ms/step - loss: 2.7089
Epoch 4/30
93/93 [==============================] - 18s 189ms/step - loss: 2.6105
Epoch 5/30
93/93 [==============================] - 18s 192ms/step - loss: 2.5429
Epoch 6/30
93/93 [==============================] - 18s 193ms/step - loss: 2.4906
Epoch 7/30
93/93 [==============================] - 18s 192ms/step - loss: 2.4369
Epoch 8/30
93/93 [==============================] - 18s 191ms/step - loss: 2.3875
Epoch 9/30
93/93 [==============================] - 18s 190ms/step - loss: 2.3420
Epoch 10/30
93/93 [==============================] - 18s 191ms/step - loss: 2.2941
Epoch 11/30
93/93 [==============================] - 18s 192ms/step - loss: 2.2499
Epoch 12/30
93/93 [==============================] - 18s 192ms/step - loss: 2.2052
Epoch 13/30
9

## 모델 평가

모델이 작문을 잘하는지 컴퓨터 알고리즘이 평가하는 것은 무리가 있다. 그래서 작문을 시켜보고 직접 평가해야 한다.

시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행하게 하는 함수generate_text를 작성한다.

In [20]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

텍스트를 생성해야 하는데 지금 source sentence와 target sentence가 없다.

이전에 테스트 데이터셋을 생성하지도 않았다.

init_sentence를 \<start\>로 시작하는 텍스트로 줘서 텍스트생성함수를 여러번 실행해보자.

In [22]:
generate_text(model, tokenizer, init_sentence="<start> he")

'<start> he hath been a suitor to the people , but that <end> '

In [23]:
generate_text(model, tokenizer, init_sentence="<start> you")

'<start> you have made a man of a <unk> , <end> '

In [24]:
generate_text(model, tokenizer, init_sentence="<start> are")

'<start> are you so hot ? <end> '